<a href="https://colab.research.google.com/github/angelesGladin/TelecomX_2/blob/main/TelecomX_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**♨️ TELECOM X - PARTE 2: PREDICCIÓN DE CANCELACIÓN (Churn)**

---
Tu nueva misión es desarrollar modelos predictivos capaces de prever qué clientes tienen mayor probabilidad de cancelar sus servicios.

La empresa quiere anticiparse al problema de la cancelación, y te corresponde a ti construir un pipeline robusto para esta etapa inicial de modelado.


##🛠️ Preparación de los Datos

In [8]:
import pandas as  pd
import warnings
warnings.filterwarnings('ignore')

datos = pd.read_csv('/content/datos_tratados.csv')
datos.sample(5)

,id_cliente,abandono,genero,es_ciudadano_senior,tiene_pareja,tiene_dependientes,meses_en_empresa,servicio_telefonico,lineas_multiples,tipo_internet,...,proteccion_dispositivo,soporte_tecnico,tv_streaming,peliculas_streaming,tipo_contrato,factura_electronica,metodo_pago,cargos_mensuales,cargos_totales,cuentas_diarias
1782,2533-QVMSK,1,Male,0,1,0,61,1,Yes,Fiber optic,...,No,Yes,Yes,No,two year,1,electronic check,94.10,5638.30,3.136667
5786,7927-AUXBZ,1,Female,0,0,0,30,1,No,Fiber optic,...,No,Yes,Yes,No,month to month,0,electronic check,85.15,2555.90,2.838333
792,1120-BMWUB,1,Female,0,0,0,16,1,No,DSL,...,No,Yes,No,No,month to month,1,mailed check,53.90,834.15,1.796667
3027,4220-TINQT,0,Female,0,1,0,61,1,Yes,Fiber optic,...,Yes,Yes,Yes,Yes,month to month,1,electronic check,106.35,6751.35,3.545000
6233,8592-PLTMQ,0,Female,0,0,0,64,1,Yes,No,...,No internet service,No internet service,No internet service,No internet service,one year,0,mailed check,24.70,1642.75,0.823333


###Eliminación de Columnas con identificador único

In [9]:
datos = datos.drop('id_cliente', axis=1)
datos.sample(2)

,abandono,genero,es_ciudadano_senior,tiene_pareja,tiene_dependientes,meses_en_empresa,servicio_telefonico,lineas_multiples,tipo_internet,seguridad_online,...,proteccion_dispositivo,soporte_tecnico,tv_streaming,peliculas_streaming,tipo_contrato,factura_electronica,metodo_pago,cargos_mensuales,cargos_totales,cuentas_diarias
1493,0,Male,0,1,1,59,1,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,two year,0,credit card automatic,19.3,1192.7,0.643333
267,1,Female,0,1,0,2,1,No,Fiber optic,Yes,...,No,No,No,No,month to month,1,electronic check,74.4,139.4,2.480000


###Codificación de las variables

In [10]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7256 entries, 0 to 7255
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   abandono                7256 non-null   int64  
 1   genero                  7256 non-null   object 
 2   es_ciudadano_senior     7256 non-null   int64  
 3   tiene_pareja            7256 non-null   int64  
 4   tiene_dependientes      7256 non-null   int64  
 5   meses_en_empresa        7256 non-null   int64  
 6   servicio_telefonico     7256 non-null   int64  
 7   lineas_multiples        7256 non-null   object 
 8   tipo_internet           7256 non-null   object 
 9   seguridad_online        7256 non-null   object 
 10  respaldo_online         7256 non-null   object 
 11  proteccion_dispositivo  7256 non-null   object 
 12  soporte_tecnico         7256 non-null   object 
 13  tv_streaming            7256 non-null   object 
 14  peliculas_streaming     7256 non-null   

Se observan variables que tienen cadenas de string por lo que se debe modificar para que sea compatibles con nuestros modelos de ML

In [11]:
#codificación de la variable binaria faltante
datos['genero'] = datos['genero'].replace({'Male':0,'Female':1})
datos['genero'].sample(2)

,genero
5442,1
1078,0


In [12]:
#codificación de las variables categóricas
categoricas = ['tipo_internet', 'seguridad_online', 'respaldo_online',
               'proteccion_dispositivo', 'soporte_tecnico', 'tv_streaming',
               'peliculas_streaming', 'tipo_contrato', 'metodo_pago',
               'lineas_multiples']

datos_codi = pd.get_dummies(data=datos,columns=categoricas, dtype=int)
datos_codi.sample(2)

,abandono,genero,es_ciudadano_senior,tiene_pareja,tiene_dependientes,meses_en_empresa,servicio_telefonico,factura_electronica,cargos_mensuales,cargos_totales,...,tipo_contrato_month to month,tipo_contrato_one year,tipo_contrato_two year,metodo_pago_bank transfer automatic,metodo_pago_credit card automatic,metodo_pago_electronic check,metodo_pago_mailed check,lineas_multiples_No,lineas_multiples_No phone service,lineas_multiples_Yes
2933,0,1,1,1,0,38,1,0,95.00,3591.25,...,0,1,0,0,1,0,0,0,0,1
6893,1,0,0,1,1,18,1,0,95.15,1779.95,...,1,0,0,0,0,0,1,1,0,0


Utilizaremos un método de pandas que nos permite codificar nuestras variables con ceros y unos de forma numérica: get_dummies. Usaremos pd.get_dummies y especificaremos nuestro conjunto de datos, las columnas que queremos codificar (columns=categóricas) y el tipo de dato (dtype) que será entero.

###Validación de la codificación de los datos

In [13]:
datos_codi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7256 entries, 0 to 7255
Data columns (total 42 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   abandono                                    7256 non-null   int64  
 1   genero                                      7256 non-null   int64  
 2   es_ciudadano_senior                         7256 non-null   int64  
 3   tiene_pareja                                7256 non-null   int64  
 4   tiene_dependientes                          7256 non-null   int64  
 5   meses_en_empresa                            7256 non-null   int64  
 6   servicio_telefonico                         7256 non-null   int64  
 7   factura_electronica                         7256 non-null   int64  
 8   cargos_mensuales                            7256 non-null   float64
 9   cargos_totales                              7256 non-null   float64
 10  cuentas_diar

En esta validación podemos observar que los datos que se encontraban con cadena de string (object) han cambiado a formato de entero (int), también se logra observar que ha crecido el número de columnas obtenidas, ya que este metodo contabiliza las opciones, por lo que ha pasado cada una de las opciones a columnas.

##SEPARANDO LAS VARIABLE DE RESPUESTA Y EXPLICATIVAS